Fetching the data from kaggle.

In [11]:

!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [12]:
!ls

sample_data


In [0]:
import json
token = {"username":"yjtrxj","key":"131bbafb775c8139a4bb07dbc9427ac7"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [15]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [17]:
!kaggle datasets list -s blog

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 51, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 910, in dataset_list_cli
    tag_ids, search, user, mine, page)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 880, in dataset_list
    return [Dataset(d) for d in datasets_list_result]
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 66, in __init__
    self.size = File.get_size(self.totalBytes)
  File "/usr/local/lib/python2.7/dist-packages/kaggle/models/kaggle_models_extended.py", line 97, in get_size
    return '%.*f%s' % (precision, size, suffixes[suffix_index])
TypeError: float argument required, not NoneType


In [18]:
!kaggle datasets download -d rtatman/blog-authorship-corpus -p /content/

 99% 288M/290M [00:03<00:00, 103MB/s] 
100% 290M/290M [00:03<00:00, 91.4MB/s]


In [19]:
!unzip \blog-authorship-corpus.zip

Archive:  blog-authorship-corpus.zip
  inflating: blogtext.csv            


In [0]:
# Importing the data.

import pandas as pd
Blog = pd.read_csv('blogtext.csv')

In [0]:
#For generating data randomly use,Blog_DF = Blog.sample(frac =0.012,replace=False,random_state=5678)

In [0]:
Blog_DF = Blog.iloc[:8000,:]

In [23]:
# Shape.

Blog_DF.shape

(8000, 7)

In [24]:
# Checking if there are any null values.

Blog_DF.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [25]:
# Looking for unique values.

Blog_DF['text'].unique()

array(['           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         ',
       '           These are the team members:   Drewes van der Laag           urlLink mail  Ruiyu Xie                     urlLink mail  Bryan Aaldering (me)          urlLink mail          ',
       "           In het kader van kernfusie op aarde:  MAAK JE EIGEN WATERSTOFBOM   How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia  Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, like gold,' as it was missing. If anyone has the full text, please distribute. I am not responsible for the accuracy of this information

In [26]:
# Checking the information.

Blog_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 7 columns):
id        8000 non-null int64
gender    8000 non-null object
age       8000 non-null int64
topic     8000 non-null object
sign      8000 non-null object
date      8000 non-null object
text      8000 non-null object
dtypes: int64(2), object(5)
memory usage: 437.6+ KB


In [27]:
Blog_DF['label'] = Blog_DF.apply(lambda r: [r['gender'], str(r['age']), r['topic'],r['sign']], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
# Dropping.

Blog_DF.drop(columns=['id','gender','age','topic','sign','date'],inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
# Verifying the change.

Blog_DF.head(2)

,text,label
0,"Info has been found (+/- 100 pages,...","[male, 15, Student, Leo]"
1,These are the team members: Drewe...,"[male, 15, Student, Leo]"


In [30]:
# Importing stop words from nltk and making some important changes.

from nltk.corpus import stopwords

import re
import nltk

nltk.download('stopwords')

Blog_DF['text'] = Blog_DF['text'].map(lambda s : s.lower())
Blog_DF['text'] = Blog_DF['text'].map(lambda s:re.sub('[0-9+-/#()!:\']','',s))
Blog_DF['text'] = Blog_DF['text'].map(lambda s :s.strip())

SW = stopwords.words('english')

Blog_DF['text'] = Blog_DF['text'].apply(lambda s: ' '.join([word for word in s.split() if word not in (SW)]))
Blog_DF['text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

0       info found pages mb pdf files wait untill team...
1       team members drewes van der laag urllink mail ...
2       het kader van kernfusie op aarde maak je eigen...
3                                         testing testing
4       thanks yahoos toolbar capture urls popupswhich...
                              ...                        
7995    today good excellent time warming becca downlo...
7996    oh goodness oh goodness oh goodness best news ...
7997    well day wasokay average long tiredblaa well e...
7998    sorry title like comment nothing entry anywayi...
7999    ready love youre sensitive pushover active man...
Name: text, Length: 8000, dtype: object

In [0]:
X = Blog_DF.text
y = Blog_DF.label

In [0]:
#A smaller train size will help the system fit in the count vectorizer.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [0]:
# Split.

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.3)

In [34]:
# Checking the shape.

print("The shape of X_train is: ",X_train.shape)
print("The shape of X_test is: ", X_test.shape)
print("The shape of X_val is: ", X_val.shape)
print("The shape of y_train is: ", y_train.shape)
print("The shape of y_test is:", y_test.shape)
print("The shape of y_val is:", y_val.shape)

The shape of X_train is:  (4480,)
The shape of X_test is:  (1600,)
The shape of X_val is:  (1920,)
The shape of y_train is:  (4480,)
The shape of y_test is: (1600,)
The shape of y_val is: (1920,)


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
CV = CountVectorizer(ngram_range = (1,2))

In [0]:
X_train = CV.fit_transform(X_train)

In [0]:
X_test = CV.transform(X_test)

In [0]:
X_val = CV.transform(X_val)

In [40]:
# Verifying.

print(X_val)

  (0, 6008)	1
  (0, 7276)	1
  (0, 7388)	1
  (0, 7847)	1
  (0, 12754)	1
  (0, 14047)	4
  (0, 14172)	1
  (0, 14206)	1
  (0, 14619)	1
  (0, 14631)	1
  (0, 15583)	1
  (0, 15756)	1
  (0, 16131)	1
  (0, 17279)	3
  (0, 17646)	1
  (0, 20114)	1
  (0, 25545)	2
  (0, 25549)	1
  (0, 28142)	1
  (0, 28625)	1
  (0, 28652)	1
  (0, 30744)	1
  (0, 33347)	1
  (0, 33770)	1
  (0, 33854)	1
  :	:
  (1918, 190418)	1
  (1918, 209108)	1
  (1918, 209207)	1
  (1918, 216400)	1
  (1918, 228002)	1
  (1918, 230926)	1
  (1918, 249946)	1
  (1918, 253048)	1
  (1918, 253049)	1
  (1918, 262773)	1
  (1918, 287146)	1
  (1918, 287399)	1
  (1919, 6612)	1
  (1919, 103472)	1
  (1919, 121977)	1
  (1919, 124307)	1
  (1919, 124666)	1
  (1919, 126670)	1
  (1919, 132298)	1
  (1919, 156979)	1
  (1919, 171076)	1
  (1919, 218047)	1
  (1919, 250400)	1
  (1919, 250436)	1
  (1919, 273589)	1


In [0]:
Label_Counts = dict()
for labels in Blog_DF.label.values:
    for label in labels:
        if label in Label_Counts:
            Label_Counts[label] += 1
        else:
            Label_Counts[label] = 1

In [43]:
Label_Counts

{'13': 9,
 '14': 170,
 '15': 390,
 '16': 73,
 '17': 914,
 '23': 144,
 '24': 378,
 '25': 268,
 '26': 112,
 '27': 691,
 '33': 101,
 '34': 540,
 '35': 2311,
 '36': 1703,
 '37': 19,
 '38': 46,
 '39': 79,
 '41': 14,
 '42': 14,
 '44': 3,
 '45': 14,
 '46': 7,
 'Accounting': 2,
 'Aquarius': 351,
 'Aries': 4140,
 'Arts': 31,
 'Automotive': 14,
 'Banking': 16,
 'BusinessServices': 87,
 'Cancer': 238,
 'Capricorn': 88,
 'Communications-Media': 61,
 'Consulting': 18,
 'Education': 121,
 'Engineering': 119,
 'Fashion': 1616,
 'Gemini': 88,
 'Internet': 93,
 'InvestmentBanking': 70,
 'Law': 3,
 'Leo': 208,
 'Libra': 425,
 'Museums-Libraries': 2,
 'Non-Profit': 47,
 'Pisces': 103,
 'Religion': 9,
 'Sagittarius': 755,
 'Science': 33,
 'Scorpio': 854,
 'Sports-Recreation': 77,
 'Student': 615,
 'Taurus': 709,
 'Technology': 2350,
 'Virgo': 41,
 'female': 3061,
 'indUnk': 2616,
 'male': 4939}

In [44]:
from sklearn.preprocessing import MultiLabelBinarizer

MLB = MultiLabelBinarizer()

y_train = MLB.fit_transform(y_train)
y_test = MLB.transform(y_test)
y_val = MLB.transform(y_val)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['Accounting', 'Museums-Libraries'] will be ignored
  .format(sorted(unknown, key=str)))


In [45]:
y_test.shape

(1600, 55)

In [46]:
y_train.shape

(4480, 55)

In [0]:
# Importing necessary models.

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

CLF = LogisticRegression(solver='lbfgs',max_iter=10000)
CLF = OneVsRestClassifier(CLF)
CLFM = OneVsRestClassifier(MultinomialNB())
CLF0 = GridSearchCV(DecisionTreeClassifier(),cv=5,param_grid={'max_depth':[80,250,500,750,1000]})
CLF1 = GridSearchCV(KNeighborsClassifier(),cv=5,param_grid={'n_neighbors':[5,10,15,20,25]})
CLF2 = GridSearchCV(RandomForestClassifier(),cv =5,param_grid={'max_depth':[80,250,500,750,1000]})


In [51]:
%%time
CLF.fit(X_train, y_train)

CPU times: user 4min 11s, sys: 6min 15s, total: 10min 27s
Wall time: 2min 43s


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=10000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred = CLF.predict(X_test)

In [53]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report

print("The model accuracy is:", CLF.score(X_train, y_train))
print("The accuracy is:", accuracy_score(y_test, y_pred))
print("The hamming-loss:", hamming_loss(y_test, y_pred))

report = classification_report(y_test,y_pred)

print(report)

The model accuracy is: 0.96875
The accuracy is: 0.3875
The hamming-loss: 0.04113636363636364
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      0.04      0.07        25
           2       0.67      0.26      0.38        76
           3       0.00      0.00      0.00        17
           4       0.78      0.31      0.44       181
           5       0.00      0.00      0.00        24
           6       0.77      0.14      0.24        71
           7       0.67      0.09      0.15        46
           8       0.00      0.00      0.00        27
           9       0.79      0.33      0.47       151
          10       1.00      0.28      0.43        18
          11       0.98      0.69      0.81       119
          12       0.75      0.71      0.73       477
          13       0.89      0.53      0.67       326
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_test_in = MLB.inverse_transform(y_test)

In [0]:
y_pred_in = MLB.inverse_transform(y_pred)

In [0]:
Label_Table = pd.DataFrame({'Predicted': y_pred_in,'Actual': y_test_in})

In [58]:
Label_Table.head(6)

,Predicted,Actual
0,"(male,)","(25, Libra, female, indUnk)"
1,"(Aries, male)","(35, Aries, Technology, male)"
2,"(17, Scorpio, female, indUnk)","(17, Scorpio, female, indUnk)"
3,"(Aries, male)","(27, Taurus, female, indUnk)"
4,"(male,)","(25, BusinessServices, Sagittarius, male)"
5,"(35, Aries, Technology, male)","(26, Gemini, indUnk, male)"


In [59]:
CLFM.fit(X_train,y_train)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True),
                    n_jobs=None)

In [0]:
y_pred1 = CLFM.predict(X_test)

In [61]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report

print("The model accuracy is:", CLFM.score(X_train, y_train))
print("The accuracy is:", accuracy_score(y_test, y_pred1))
print("The hamming-loss:", hamming_loss(y_test, y_pred1))

report1 = classification_report(y_test, y_pred1)

print(report1)

The model accuracy is: 0.7805803571428571
The accuracy is: 0.06875
The hamming-loss: 0.054113636363636364
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        25
           2       0.67      0.05      0.10        76
           3       0.00      0.00      0.00        17
           4       1.00      0.05      0.09       181
           5       0.00      0.00      0.00        24
           6       1.00      0.03      0.05        71
           7       0.00      0.00      0.00        46
           8       0.00      0.00      0.00        27
           9       1.00      0.01      0.01       151
          10       0.00      0.00      0.00        18
          11       0.00      0.00      0.00       119
          12       0.93      0.09      0.16       477
          13       0.97      0.19      0.32       326
          14       0.00      0.00      0.00         4
          15       0.00      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
CLF1.fit(X_train, y_train)

print("This is the best score :", CLF1.best_score_)
print("These are the best parameters:", CLF1.best_params_)
print("This is the best estimator:", CLF1.best_estimator_)

This is the best score : 0.08839285714285713
These are the best parameters: {'n_neighbors': 5}
This is the best estimator: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [63]:
LR1 = CLF1.best_estimator_

LR1.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
y_pred2 = LR1.predict(X_test)

In [65]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report

print("The model accuracy is: ", LR1.score(X_train, y_train))
print("The accuracy is: ", accuracy_score(y_test,y_pred2))
print("The hamming-loss: ", hamming_loss(y_test, y_pred2))

report2 = classification_report(y_test,y_pred2)
print(report2)

The model accuracy is:  0.17611607142857144
The accuracy is:  0.10125
The hamming-loss:  0.08956818181818182
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        25
           2       0.38      0.04      0.07        76
           3       0.00      0.00      0.00        17
           4       0.09      0.22      0.12       181
           5       0.00      0.00      0.00        24
           6       1.00      0.03      0.05        71
           7       0.00      0.00      0.00        46
           8       0.00      0.00      0.00        27
           9       0.33      0.01      0.01       151
          10       0.00      0.00      0.00        18
          11       0.67      0.02      0.03       119
          12       0.22      0.25      0.24       477
          13       0.61      0.03      0.06       326
          14       0.00      0.00      0.00         4
          15       0.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred_re0 = MLB.inverse_transform(y_pred2)

In [0]:
Label_Table0 = pd.DataFrame({'Predicted':y_pred_re0,'Actual': y_test_in})

In [68]:
Label_Table0.head(6)

,Predicted,Actual
0,"(Aries, male)","(25, Libra, female, indUnk)"
1,"(35, Aries, Technology, male)","(35, Aries, Technology, male)"
2,"(17, Scorpio, female, indUnk)","(17, Scorpio, female, indUnk)"
3,"(17, female)","(27, Taurus, female, indUnk)"
4,"(Aries, male)","(25, BusinessServices, Sagittarius, male)"
5,"(35, Aries, Technology, male)","(26, Gemini, indUnk, male)"


In [69]:
X_train.shape

(4480, 300716)

In [70]:
y_train.shape

(4480, 55)

In [71]:
CLF2.fit(X_train, y_train)

print("The best score is:", CLF2.best_score_)
print("These are the best parameters:", CLF2.best_params_)
print("This is the best estimator:", CLF2.best_estimator_)

The best score is: 0.26941964285714287
These are the best parameters: {'max_depth': 500}
This is the best estimator: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [72]:
LR2 = CLF2.best_estimator_

LR2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred3 = LR2.predict(X_test)

In [75]:
from sklearn.metrics import hamming_loss, accuracy_score,classification_report

print("The model accuracy is:", LR2.score(X_train,y_train))
print("The accuracy is:", accuracy_score(y_test,y_pred3))
print("The hamming-loss:", hamming_loss(y_test, y_pred3))

report3 = classification_report(y_test,y_pred3)

print(report3)

The model accuracy is: 0.9830357142857142
The accuracy is: 0.274375
The hamming-loss: 0.05703409090909091
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        25
           2       0.00      0.00      0.00        76
           3       0.00      0.00      0.00        17
           4       1.00      0.01      0.02       181
           5       0.00      0.00      0.00        24
           6       1.00      0.03      0.05        71
           7       1.00      0.02      0.04        46
           8       0.00      0.00      0.00        27
           9       1.00      0.01      0.01       151
          10       0.00      0.00      0.00        18
          11       1.00      0.02      0.03       119
          12       0.61      0.81      0.70       477
          13       0.95      0.17      0.28       326
          14       0.00      0.00      0.00         4
          15       0.00      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_pred_re1 = MLB.inverse_transform(y_pred3)

In [0]:
Label_Table1 = pd.DataFrame({'Predicted':y_pred_re1,'Actual':y_test_in})

In [88]:
Label_Table1.head(4)

,Predicted,Actual
0,"(male,)","(25, Libra, female, indUnk)"
1,"(35, Aries, Technology, male)","(35, Aries, Technology, male)"
2,"(female, indUnk)","(17, Scorpio, female, indUnk)"
3,"(35, Aries, Technology, male)","(27, Taurus, female, indUnk)"


Conclusion : The model accuracy achieved in training the models is decent. These models don't perform twhen they are put to test. 
